# EE 120 Lab 7: Communication

**Signals and Systems** at UC Berkeley

Acknowledgements:
* **Spring 2020** (v1.0): Jonathan Lee, Galen Kimball, Babak Ayazifar

## Introduction

You may not realize it, but the air is filled with radio waves, an invisible form of electromagnetic radiation.
Your laptop's wifi card, TV stations, airplane transceivers, and even the [International Space Station](https://spaceflight.nasa.gov/station/reference/radio/index.html) all use radio to carry information at the speed of light.
In this lab, we'll explore several digital communication schemes in which a sender transmits bits to a receiver over a *channel*, a physical medium like radio.

### Dependencies

We will reuse the [`sounddevice`](https://python-sounddevice.readthedocs.io/en/0.3.15/) package from earlier labs to play and record audio simultaneously.
Install `sounddevice` using the Python package manager of your choice (we recommend [`pip`](https://pip.pypa.io/en/stable/installing/)):
```
$ pip install --user sounddevice
$ conda install -c conda-forge python-sounddevice
```
To get assistance with installation on Piazza, provide as much debug information as you can, including operating system, Python version, steps to reproduce the error, and the full traceback.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import cluster, signal
from scipy.io import wavfile
import sounddevice as sd

if 'ggplot' in plt.style.available:
    plt.style.use('ggplot')

%matplotlib inline

## Q1: Amplitude Modulation

First, we'll review classic analog amplitude modulation (AM).
Suppose Babak wants to share an audio recording of one of his lectures, a bandlimited continuous-time signal $x(t)$, with the class.
Unfortunately, he cannot efficiently transmit $x(t)$ as-is across long distances because the human voice is simply not energetic enough.

AM solves this problem by layering $x(t)$ on a purely sinusoidal *carrier signal*
$$c(t) = \cos{\omega_0 t} \stackrel{\mathcal{F}}{\longleftrightarrow} \pi \left[\delta(\omega + \omega_0) + \delta(\omega - \omega_0)\right]$$
using multiplication, which effectively shapes $c(t)$'s envelope:
$$r(t) \triangleq x(t) c(t) \stackrel{\mathcal{F}}{\longleftrightarrow} R(\omega) = \frac{1}{2\pi} (X * C)(\omega) = \frac{1}{2} \left[X(\omega+\omega_0) + X(\omega-\omega_0)\right]$$
Typically, the carrier frequency $\omega_0$ is much greater than the maximum frequency in $x(t)$.

After $r(t)$ is transmitted and received, the students in the class recover $x(t)$ by demodulating, which entails lowpass filtering $r(t) c(t)$:
$$r(t) c(t) \stackrel{\mathcal{F}}{\longleftrightarrow} \frac{1}{2\pi} (R * C)(\omega) = \underbrace{\frac{1}{4} X(\omega+2\omega_0)}_{\text{Removed}} + \underbrace{\frac{1}{2} X(\omega)}_{\text{Amplify by } 2} + \underbrace{\frac{1}{4} X(\omega-2\omega_0)}_{\text{Removed}}$$

In [ ]:
def play(signal):
    try:
        sd.play(signal, blocking=True)
    except KeyboardInterrupt:
        print('Interrupted playback.')

First, we'll load the recording that Babak wants to send.
If you check the shape of `stereo`, you'll notice the lecture recording has two channels that are fed into the left and right sides of your headphones/speaker.

**Your task**: Since we don't care about this feature, average the two channels using [`np.mean`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) to convert the recording into mono (1D) format, then listen to the recording.
You can interrupt a playback using "Kernel > Interrupt" in the Jupyter notebook menu.

In [ ]:
fs, stereo = wavfile.read('babak.wav')
print('Original shape:', stereo.shape)
mono = ...  # *** TODO: Your code here ***
mono = mono.astype(np.float32)/np.iinfo(np.int16).max
sd.default.samplerate = fs
sd.default.channels = 1
play(mono)

**Your task**: In the next cell, implement `am_modulate` according to the docstring.

We modulate to $\omega_0 = \pm 8~\text{kHz}$ and play the recording.
It should sound unintelligible and somewhat scratchy because everything has been moved to higher frequencies.

In [ ]:
def am_modulate(x, f0=8e3, fs=48e3):
    """
    Amplitude modulate the given baseband signal.
    
    Arguments:
        x (numpy.ndarray): Samples of the baseband signal x(t), as
            defined in the derivation above, sampled at the given
            sampling rate.
        f0 (float): Carrier frequency in Hertz. Remember to multiply
            by 2\pi to get a radial frequency \omega_0 in rad/s!
        fs (float): Sampling frequency in Hertz.
    
    Hint:
        Use the sampling frequency and `len(x)`, the number of samples,
        to generate the sampling times `t`. Once you have `t`, generate
        the carrier sinusoid.
    """
    # *** TODO: Your code here ***

modulated = am_modulate(mono, fs=fs)
play(modulated)

**Your task:** Implement `am_demodulate` (similar to `am_modulate`).

We demodulate *without* applying a lowpass filter (i.e, there are replicas of the baseband centered at $\pm 16~\text{kHz}$).
The received recording is intelligble, but you can also hear the unpleasant high frequencies.

In [ ]:
def am_demodulate(r, f0=8e3, fs=48e3, lpf=None):
    """
    Recover the baseband signal from an amplitude-modulated signal.
    
    Arguments:
        r: Received modulated signal.
        f0: Carrier frequency.
        fs: Sampling frequency.
        lpf (numpy.ndarray): The impulse response of the AM lowpass filter
            (optional). Is `None` when not provided.
    
    Note::
        Remember that the final result needs to be scaled by 2.
    """
    # *** TODO: Your code here ***

demodulated_with_replicas = am_demodulate(modulated)
play(demodulated_with_replicas)

**Your task**: In the next cell, design the impulse response of the AM lowpass filter (LPF) using [`scipy.signal.firwin`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html).
We recommend `numtaps` (i.e., impulse response length) of ~30.
The lowpass filter magnitude response, modulated spectrum, and baseband spectrum are plotted below, and can be used to help you pick a suitable cutoff frequency.

Recall that the ideal lowpass filter has a rectangular spectrum but is IIR, which not only requires infinite memory, but is not causal.
One way to design an FIR, causal LPF is to "window" (truncate) and delay the impulse response.
In the frequency domain, this corresponds to convolving the filter magnitude response with the window's transform, then multiply by a complex exponential (time delay property).
The convolution with the window's transform results in the curved shape that you see.

In [ ]:
def normalized_spectrum(x):
    """
    Compute the single-sided spectrum (magnitude) of a real signal,
    normalizing to one.
    """
    X = np.abs(np.fft.rfft(x))
    return X/X.max()

lpf = ...  # *** TODO: Your code here ***
w, h_lpf = signal.freqz(lpf, fs=fs)

h_mono = normalized_spectrum(mono)
h_mod = normalized_spectrum(modulated)

w_ss = np.linspace(0, 0.5*fs, h_mono.size)
plt.figure(figsize=(10, 6))
plt.plot(w, np.abs(h_lpf), color='tab:red', linestyle='dashed')
plt.plot(w_ss, h_mono, label='Original', color='tab:blue', alpha=0.3)
plt.plot(w_ss, h_mod, label='Amplitude-Modulated', color='tab:orange', alpha=0.3)
plt.legend()

The next block demodulates *and* applies the lowpass filter, which removes the high frequency replicas.
You can also see that the mean squared error (MSE) is small relative to the maximum amplitude of the mono signal (~1).

In [ ]:
demodulated = am_demodulate(modulated, lpf=lpf)
print('MSE: {}'.format(np.round(np.mean((demodulated - mono)**2), 5)))
play(demodulated)

## Q2: On-Off Keying (OOK)

We turn our attention to a digital communication scheme, on-off keying.
The idea is fairly straightforward: transmit the carrier for the "one" bit, and nothing for the "zero" bit.
We can view this as a special case of AM with a piecewise-constant $x(t) \in \{0, 1\}$.
Bits are transmitted according to an agreed-upon bit rate.

Because real hardware cannot produce discontinuous signals, we shouldn't jump straight to or from zero during a bit transition.
Instead, every bit period should be windowed with a [Hann window](https://en.wikipedia.org/wiki/Hann_function), which has the effect of tapering the edges to zero and producing a continuous "pulse".

**Your task:** Implement the function `ook_encode` using `am_modulate`. Consult the [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.hann.html?highlight=hann#scipy.signal.windows.hann) to generate the window.

We'll plot the modulated signal and its spectrum for a short bitstream.

In [ ]:
def ook_encode(bits, bit_rate=200, f0=8e3, fs=48e3):
    """
    Encode a packet of bits using on-off keying with Hann windowing.
    
    Arguments:
        bits (numpy.ndarray): 1D array of bits (0 or 1).
        bit_rate (float): Frequency bits are sent at.
        f0 (float): Carrier frequency.
        fs (float): Sampling frequency.
    
    All frequencies are in Hertz.
    
    Hint:
        Assume the sampling rate is evenly divisible by the bit rate.
        Then, the quantity `fs/bit_rate` is an integer, and can be
        interpreted as the number of samples per bit.
    """
    # *** TODO: Your code here ***

packet = np.array([0, 1, 0, 0, 1, 1], dtype=np.uint8)
ook_baseband = ook_encode(packet)
plt.figure(figsize=(10, 6))
plt.plot(ook_baseband)
plt.figure(figsize=(10, 6))
plt.plot(np.linspace(-0.5*fs, 0.5*fs, ook_baseband.size),
         np.abs(np.fft.fft(ook_baseband)))